In [1]:
import pandas as pd
import scipy.stats as stats
import cptac.utils as ut
import scipy
import statsmodels.stats.multitest as ssm
import numpy as np
import sys, os
currentdir = os.path.dirname(os.path.realpath('make_regression_df.py'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
import Delta_Correlation as dc

In [2]:
cancers, cancer_names = dc.load_cancers()

cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps\Delta_Correlation.py, line 14)


cptac warning: The pdac data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps\Delta_Correlation.py, line 15)


In [3]:
def log2_fold_change(tumor, normal):
    tumor = np.mean(tumor)
    normal = np.mean(normal)
    if normal == 0 or tumor == 0:
        return(float('Nan'))
    fold_change = tumor / normal
    fold_change = abs(fold_change)
    log2_change = np.log2(fold_change)
    return log2_change    

In [4]:
diff_expression_dfs = []
for cancer, cancer_name in zip(cancers, cancer_names):
    prot_df = dc.get_prot_trans_df(cancer)
    prot_df = prot_df.drop(columns = 'Transcriptomics')
    fold_change = prot_df.groupby('Gene').apply(
        lambda df: log2_fold_change(df[df['Tissue']=='Tumor'].Proteomics,
                                    df[df['Tissue']=='Normal'].Proteomics))
    fold_change = pd.DataFrame(fold_change, columns = ['Log2_fold_change'])
    fold_change.reset_index(inplace = True)
    ranksums = prot_df.groupby('Gene').apply(lambda df: stats.ranksums(
        df[df['Tissue']=='Tumor'].Proteomics,
        df[df['Tissue']=='Normal'].Proteomics))
    ranksums = pd.DataFrame.from_records(ranksums, index = ranksums.index, columns = ['statistic', 'pval'])
    ranksums.reset_index(inplace = True)
    ranksums = ranksums.dropna()
    ranksums['FDR'] = ssm.fdrcorrection(ranksums['pval'])[1]
    ranksums['Cancer'] = [cancer_name] * len(ranksums)
    diff_expression_df = pd.merge(ranksums, fold_change)
    diff_expression_dfs.append(diff_expression_df)
diff_expression_df = pd.concat(diff_expression_dfs)
diff_expression_df.to_csv('Proteomics_differential_expression_df.csv', index = False)
diff_expression_df

cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps\Delta_Correlation.py, line 23)
cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps\Delta_Correlation.py, line 33)
C:\Users\nanel\anaconda3\lib\site-packages\scipy\stats\stats.py:7114: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
C:\Users\nanel\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\nanel\anaconda3\lib\site-packages\scipy\stats\stats.py:7114: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
C:\Users\nanel\anaconda3\lib\site-packages\scipy\stats\_dist

,Gene,statistic,pval,FDR,Cancer,Log2_fold_change
0,A1BG,3.652165,2.600390e-04,4.129259e-04,CCRCC,1.095344
1,A1CF,-6.297607,3.022755e-10,6.980369e-10,CCRCC,-0.001629
2,A2M,6.104652,1.030249e-09,2.307942e-09,CCRCC,-1.195447
3,A4GALT,0.979796,3.271869e-01,3.669641e-01,CCRCC,3.045874
4,AAAS,8.984996,2.587430e-19,1.009474e-18,CCRCC,4.400288
...,...,...,...,...,...,...
11437,ZXDC,1.889822,5.878172e-02,1.457034e-01,PDAC,0.068755
11438,ZYG11B,1.148810,2.506345e-01,4.076418e-01,PDAC,0.005405
11439,ZYX,3.779675,1.570334e-04,1.621639e-03,PDAC,0.014651
11440,ZZEF1,-0.527046,5.981615e-01,7.311359e-01,PDAC,-0.001804


In [5]:
diff_expression_dfs = []
for cancer, cancer_name in zip(cancers, cancer_names):
    prot_df = dc.get_prot_trans_df(cancer)
    prot_df = prot_df.drop(columns = 'Proteomics')
    fold_change = prot_df.groupby('Gene').apply(
        lambda df: log2_fold_change(df[df['Tissue']=='Tumor'].Transcriptomics,
                                    df[df['Tissue']=='Normal'].Transcriptomics))
    fold_change = pd.DataFrame(fold_change, columns = ['Log2_fold_change'])
    fold_change.reset_index(inplace = True)
    ranksums = prot_df.groupby('Gene').apply(lambda df: stats.ranksums(
        df[df['Tissue']=='Tumor'].Transcriptomics,
        df[df['Tissue']=='Normal'].Transcriptomics))
    ranksums = pd.DataFrame.from_records(ranksums, index = ranksums.index, columns = ['statistic', 'pval'])
    ranksums.reset_index(inplace = True)
    ranksums = ranksums.dropna()
    ranksums['FDR'] = ssm.fdrcorrection(ranksums['pval'])[1]
    ranksums['Cancer'] = [cancer_name] * len(ranksums)
    diff_expression_df = pd.merge(ranksums, fold_change)
    diff_expression_dfs.append(diff_expression_df)
diff_expression_df = pd.concat(diff_expression_dfs)
diff_expression_df.to_csv('Transcriptomics_differential_expression_df.csv', index = False)
diff_expression_df

cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps\Delta_Correlation.py, line 23)
cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (C:\Users\nanel\Documents\GitHub\pancancerProteinMRNA\notebook_steps\Delta_Correlation.py, line 33)
C:\Users\nanel\anaconda3\lib\site-packages\scipy\stats\stats.py:7114: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
C:\Users\nanel\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
C:\Users\nanel\anaconda3\lib\site-packages\scipy\stats\stats.py:7114: RuntimeWarning: invalid value encountered in double_scalars
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)
C:\Users\nanel\anaconda3\lib\site-packages\scipy\stats\_dist

,Gene,statistic,pval,FDR,Cancer,Log2_fold_change
0,A1BG,6.602420,4.044989e-11,9.864646e-11,CCRCC,0.881967
1,A1CF,-5.201398,1.977946e-07,3.785302e-07,CCRCC,-0.641470
2,A2M,5.970422,2.366402e-09,5.132955e-09,CCRCC,0.514646
3,A4GALT,0.244949,8.064959e-01,8.241685e-01,CCRCC,0.096223
4,AAAS,1.596773,1.103162e-01,1.296881e-01,CCRCC,0.078019
...,...,...,...,...,...,...
11437,ZXDC,-0.629941,5.287333e-01,6.351461e-01,PDAC,-0.009334
11438,ZYG11B,-0.720238,4.713784e-01,5.830193e-01,PDAC,-0.006022
11439,ZYX,2.730602,6.321883e-03,1.687704e-02,PDAC,0.043200
11440,ZZEF1,-6.806426,1.000530e-11,2.725730e-09,PDAC,-0.087991
